In [1]:
from model import AdaptationModel
from mesa.batchrunner import batch_run
import pandas as pd

In [ ]:
#Extreme values test
m1 = AdaptationModel(number_of_households=100, income_distribution={'Poor':[0,0],'Middle-Class':[0,0],'Rich':[0,0]})
m1.model_run()

C:\Users\matth\OneDrive\Documenten\GitHub\SEN1211-ABM\Interpreter\Lib\site-packages\mesa\time.py:80: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


In [3]:
def unpack_dictionary(row, column_name):
    return pd.Series(row[column_name])

#base_case_unpacked = base_case_100_iterations.apply(unpack_dictionary['AverageDamagePerIncomeLabel'], axis = 1)
first_column_unpacked = model_vars_extreme_values_df.apply(unpack_dictionary, args=('AverageDamagePerIncomeLabel',), axis=1)
second_column_unpacked = model_vars_extreme_values_df.apply(unpack_dictionary, args=('AverageIncomeToDamageRatio',), axis=1)
extreme_values_unpacked = pd.concat([model_vars_extreme_values_df, first_column_unpacked, second_column_unpacked], axis=1)
model_vars_extreme_values_clean = extreme_values_unpacked.drop(['AverageDamagePerIncomeLabel', 'AverageIncomeToDamageRatio'], axis=1)
model_vars_extreme_values_clean

,RunId,iteration,Step,number_of_households,number_of_steps,flood_map_choice,network,TotalAdaptedHouseholds,TotalActualDamage,TotalExpectedDamage,TotalAdaptationCosts,TotalCostsOfSubsidies,AverageDamagePerPoorHousehold,AverageDamagePerMiddleClassHousehold,AverageDamagePerRichHousehold,AverageIncomeToDamagePoorHousehold,AverageIncomeToDamageMiddleClassHousehold,AverageIncomeToDamageRichHousehold
0,0,0,14,1000,80,harvey,watts_strogatz,544,4.709781e+07,4.972030e+07,2075365,0,22715.367157,48211.893268,104519.743342,4.612960,1.644535,1.199793
1,2,0,31,1000,80,500yr,watts_strogatz,417,2.222798e+07,2.379461e+07,1844925,0,11900.904269,20983.909927,48968.959331,2.351506,0.721397,0.543312
2,1,0,48,1000,80,100yr,watts_strogatz,317,1.372802e+07,1.480638e+07,1600920,0,6638.366233,14334.351130,26910.397905,1.407224,0.476962,0.308374
3,5,1,16,1000,80,500yr,watts_strogatz,384,2.010436e+07,2.154321e+07,1793980,0,7851.177012,20469.100865,49965.909125,1.572970,0.680746,0.575159
4,3,1,53,1000,80,harvey,watts_strogatz,589,4.777509e+07,5.050010e+07,4701615,0,26022.510915,45162.400866,109430.351939,5.342378,1.561452,1.254880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,98,55,1000,80,500yr,watts_strogatz,357,2.137778e+07,2.265748e+07,2151075,0,9590.297553,22575.464957,43941.022307,1.915211,0.768634,0.503647
296,297,99,16,1000,80,harvey,watts_strogatz,586,4.297254e+07,4.600827e+07,2721615,0,22367.835930,42564.368419,106320.839645,4.527344,1.448382,1.204534
297,295,98,69,1000,80,100yr,watts_strogatz,314,1.372589e+07,1.465051e+07,2119400,0,7946.539588,13327.761127,31548.537921,1.536154,0.461211,0.360239
298,298,99,42,1000,80,100yr,watts_strogatz,276,1.412606e+07,1.517185e+07,1537025,0,7588.228615,13499.524720,34157.269477,1.450464,0.454982,0.400971


In [4]:
output_data_path = r'../output_data/ExtremeValuesModelVars.csv'
model_vars_extreme_values_clean.to_csv(output_data_path, index=False)